In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you prolly won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, '../../venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

# so that we can import `tools.py`
# Navigate two directories up from the current working directory
module_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
# Add this directory to sys.path if it's not already included
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import pickle
from tqdm import tqdm

In [3]:
# load the dataset
with open('../../input.txt', 'r', encoding='utf-8') as f:
    tinyShakespeare_string = f.read()

In [4]:
# Convert the string to bytes
tinyShakespeare_bytes = tinyShakespeare_string.encode('utf-8')

print(tinyShakespeare_string[:100])
print([b for b in tinyShakespeare_bytes[:100]])
print(tinyShakespeare_bytes.decode('utf-8')[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
[70, 105, 114, 115, 116, 32, 67, 105, 116, 105, 122, 101, 110, 58, 10, 66, 101, 102, 111, 114, 101, 32, 119, 101, 32, 112, 114, 111, 99, 101, 101, 100, 32, 97, 110, 121, 32, 102, 117, 114, 116, 104, 101, 114, 44, 32, 104, 101, 97, 114, 32, 109, 101, 32, 115, 112, 101, 97, 107, 46, 10, 10, 65, 108, 108, 58, 10, 83, 112, 101, 97, 107, 44, 32, 115, 112, 101, 97, 107, 46, 10, 10, 70, 105, 114, 115, 116, 32, 67, 105, 116, 105, 122, 101, 110, 58, 10, 89, 111, 117]
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [5]:
print(set(tinyShakespeare_bytes))

{10, 32, 33, 36, 38, 39, 44, 45, 46, 51, 58, 59, 63, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122}


# Regex
this is a pre-processing stage where we set the rules for what types of characters are allowed to be merged together

In [7]:
import regex as re

In [8]:
# don't ask me the specifics of how this plays out, i just know it's what they used for GPT4
GPT4_SPLIT_PATTERN = r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+"""
# if you want to mess around with building your own tokenizer, then ^this string is one of the things to mess around with

In [9]:
compiled_pattern = re.compile(GPT4_SPLIT_PATTERN, re.UNICODE)
print(compiled_pattern)

regex.Regex("'(?i:[sdmt]|ll|ve|re)|[^\\r\\n\\p{L}\\p{N}]?+\\p{L}+|\\p{N}{1,3}| ?[^\\s\\p{L}\\p{N}]++[\\r\\n]*|\\s*[\\r\\n]|\\s+(?!\\S)|\\s+", flags=regex.V0)


In [10]:
# split the text up into text chunks
text_chunks = re.findall(compiled_pattern, tinyShakespeare_string)
print(len(tinyShakespeare_string), len(text_chunks))

1115394 263198


In [11]:
print(tinyShakespeare_string[:100])
print(text_chunks[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You
['First', ' Citizen', ':\n', 'Before', ' we', ' proceed', ' any', ' further', ',', ' hear', ' me', ' speak', '.\n\n', 'All', ':\n', 'Speak', ',', ' speak', '.\n\n', 'First', ' Citizen', ':\n', 'You', ' are', ' all', ' resolved', ' rather', ' to', ' die', ' than', ' to', ' famish', '?\n\n', 'All', ':\n', 'Resolved', '.', ' resolved', '.\n\n', 'First', ' Citizen', ':\n', 'First', ',', ' you', ' know', ' Caius', ' Marcius', ' is', ' chief', ' enemy', ' to', ' the', ' people', '.\n\n', 'All', ':\n', 'We', ' know', "'t", ',', ' we', ' know', "'t", '.\n\n', 'First', ' Citizen', ':\n', 'Let', ' us', ' kill', ' him', ',', ' and', ' we', "'ll", ' have', ' corn', ' at', ' our', ' own', ' price', '.\n', 'Is', "'t", ' a', ' verdict', '?\n\n', 'All', ':\n', 'No', ' more', ' talking', ' on', "'t", ';', ' let', ' it', ' be', ' done']


In [12]:
# input text preprocessing
ids = [list(ch.encode("utf-8")) for ch in text_chunks] 
ids_backup = ids # saving this for later just to see how much compression we get
print(len(ids), ids[:100])

263198 [[70, 105, 114, 115, 116], [32, 67, 105, 116, 105, 122, 101, 110], [58, 10], [66, 101, 102, 111, 114, 101], [32, 119, 101], [32, 112, 114, 111, 99, 101, 101, 100], [32, 97, 110, 121], [32, 102, 117, 114, 116, 104, 101, 114], [44], [32, 104, 101, 97, 114], [32, 109, 101], [32, 115, 112, 101, 97, 107], [46, 10, 10], [65, 108, 108], [58, 10], [83, 112, 101, 97, 107], [44], [32, 115, 112, 101, 97, 107], [46, 10, 10], [70, 105, 114, 115, 116], [32, 67, 105, 116, 105, 122, 101, 110], [58, 10], [89, 111, 117], [32, 97, 114, 101], [32, 97, 108, 108], [32, 114, 101, 115, 111, 108, 118, 101, 100], [32, 114, 97, 116, 104, 101, 114], [32, 116, 111], [32, 100, 105, 101], [32, 116, 104, 97, 110], [32, 116, 111], [32, 102, 97, 109, 105, 115, 104], [63, 10, 10], [65, 108, 108], [58, 10], [82, 101, 115, 111, 108, 118, 101, 100], [46], [32, 114, 101, 115, 111, 108, 118, 101, 100], [46, 10, 10], [70, 105, 114, 115, 116], [32, 67, 105, 116, 105, 122, 101, 110], [58, 10], [70, 105, 114, 115, 116], [

so this regex just splits the text up into all the token ids that are allowed to be merged, meaning that the regex output we saw above is an upper limit on the tokens that we could end up with if we get a large enough vocabulary, rather than a starting point. 

# BPE tokenization

In [15]:
vocab_size = 2048 - 3 # the desired final vocabulary size. -3 for the three special tokens bos, eos, & pad
num_merges = vocab_size - 256

In [16]:
def get_stats(ids, counts=None):
    """
    Given a list of integers, return a dictionary of counts of consecutive pairs
    Example: [1, 2, 3, 1, 2] -> {(1, 2): 2, (2, 3): 1, (3, 1): 1}
    Optionally allows to update an existing dictionary of counts
    """
    counts = {} if counts is None else counts
    for pair in zip(ids, ids[1:]): # iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
    """
    In the list of integers (ids), replace all consecutive occurrences
    of pair with the new integer token idx
    Example: ids=[1, 2, 3, 1, 2], pair=(1, 2), idx=4 -> [4, 3, 4]
    """
    newids = []
    i = 0
    while i < len(ids):
        # if not at the very last position AND the pair matches, replace it
        if ids[i] == pair[0] and i < len(ids) - 1 and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

In [17]:
# now let's actually do it
merges = {} # (int, int) -> int
for i in range(num_merges):#tqdm(range(num_merges)):
    # count the number of times every consecutive pair appears
    stats = {}
    for chunk_ids in ids:
        # passing in stats will update it in place, adding up counts
        get_stats(chunk_ids, stats)
    # find the pair with the highest count
    pair = max(stats, key=stats.get)
    # mint a new token: assign it the next available id
    idx = 256 + i
    # replace all occurrences of pair in ids with idx
    ids = [merge(chunk_ids, pair, idx) for chunk_ids in ids]
    # save the merge
    merges[pair] = idx
    print(f"merge {i+1}/{num_merges}: {pair} -> {idx} had {stats[pair]} occurrences")

merge 1/1789: (32, 116) -> 256 had 23837 occurrences
merge 2/1789: (104, 101) -> 257 had 18203 occurrences
merge 3/1789: (32, 97) -> 258 had 13541 occurrences
merge 4/1789: (111, 117) -> 259 had 12730 occurrences
merge 5/1789: (32, 115) -> 260 had 12287 occurrences
merge 6/1789: (32, 109) -> 261 had 10786 occurrences
merge 7/1789: (105, 110) -> 262 had 10606 occurrences
merge 8/1789: (32, 119) -> 263 had 10546 occurrences
merge 9/1789: (114, 101) -> 264 had 9843 occurrences
merge 10/1789: (104, 97) -> 265 had 9673 occurrences
merge 11/1789: (58, 10) -> 266 had 8762 occurrences
merge 12/1789: (110, 100) -> 267 had 8730 occurrences
merge 13/1789: (256, 257) -> 268 had 8684 occurrences
merge 14/1789: (32, 98) -> 269 had 8463 occurrences
merge 15/1789: (105, 115) -> 270 had 7526 occurrences
merge 16/1789: (111, 114) -> 271 had 7297 occurrences
merge 17/1789: (10, 10) -> 272 had 7098 occurrences
merge 18/1789: (32, 102) -> 273 had 6563 occurrences
merge 19/1789: (101, 114) -> 274 had 6515 o

In [32]:
og = sum([len(t) for t in ids_backup])
new = sum([len(t) for t in (ids)])
print("original length:", og)
print("ids length:", new)
print(f"compression ratio: {og / new:.2f}X")

original length: 1115394
ids length: 355780
compression ratio: 3.14X


In [38]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    # given ids (list of integers), return Python string
    tokens = b"".join(vocab[idx] for idx in ids)
    text = tokens.decode("utf-8", errors="replace")
    return text

print(ids[0], decode(ids[0]))

[681] First


In [40]:
def encode(text):
    # given a string, return list of integers (the tokens)
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        if pair not in merges:
            break # nothing else can be merged
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

print(encode("JULIET:\nO Romeo, Romeo! wherefore art thou"))

[1017, 266, 79, 1179, 44, 1179, 33, 717, 558, 752, 348, 433]


In [50]:
# Ensure the tokenizers directory exists
if not os.path.exists('./models'):
    os.makedirs('./models')

# Prepare the tokenizer data to be saved
tokenizer_data = {'merges': merges}

# Save the tokenizer data using pickle
with open(f'./models/{vocab_size}.model', 'wb') as f:
    pickle.dump(tokenizer_data, f)

In [48]:
# taking a pre-existing tokenizer and trimming it down to a smaller size
# i basically ran this cell and then one above it multiple times until i got to the smallest possible size (512 - 3 = 509)
vocab_size = ((vocab_size + 3) // 2) - 3 # the -3's account for our special tokens
merges = {k: v for k, v in merges.items() if v < vocab_size}

In [52]:
from tokenizer import load_tokenizer_data, BPE_Tokenizer
vocab_size = 2048
tokenizer_data = load_tokenizer_data(f'models/{vocab_size-3}.model')
tokenizer = BPE_Tokenizer(tokenizer_data['merges'])

In [56]:
prompt = 'JULIET:\nO Romeo, Romeo! wherefore art thou'
print(tokenizer.display(prompt))

tokens = tokenizer.encode(prompt)
print(tokens)

decoded = tokenizer.decode(tokens)
print(decoded)

['JULIET', ':\n', 'O', ' Romeo', ',', ' Romeo', '!', ' where', 'fore', ' art', ' thou']
[2045, 1017, 266, 79, 1179, 44, 1179, 33, 717, 558, 752, 348]
JULIET:
O Romeo, Romeo! wherefore art thou
